## Code for sweeping angle and getting voltage

In [ ]:
#!/usr/bin/env python3
import time
import numpy as np
import serial
from redpitaya_scpi import scpi
from redpitaya_acquirer import RedPitayaAcquirer

class StageController:
    """
    Minimal ELL14K controller for relative/absolute moves.
    """
    def __init__(self, port, baudrate=9600, timeout=1.0):
        self.ser = serial.Serial(
            port=port,
            baudrate=baudrate,
            bytesize=serial.EIGHTBITS,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            timeout=timeout
        )
        # Flush buffers
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()
        time.sleep(0.1)
        self.pulses_per_deg = 262144 / 360.0

    def move_absolute(self, deg: float) -> str:
        pulses = int(deg * self.pulses_per_deg)
        cmd = f"0ma{pulses:08X}\r".encode("ascii")
        self.ser.write(cmd)
        return self.ser.readline().decode("ascii").strip()

    def close(self):
        try:
            self.ser.close()
        except:
            pass

def find_min_voltage_angle(
    pitaya_ip: str,
    pitaya_port: int,
    com_port: str,
    angle_start: float = 0.0,
    angle_end: float = 360.0,
    angle_step: float = 1.0,
    measure_duration: float = 2.0
):
    """
    Sweeps the stage from angle_start to angle_end in steps of angle_step,
    measures CH1 mean voltage at each angle, and prints the angle with the minimum voltage.
    """
    # 1) Connect to instruments
    rp = RedPitayaAcquirer(ip=pitaya_ip, port=pitaya_port, decimation=1024, avg_off=True)
    stage = StageController(port=com_port, baudrate=9600)

    try:
        angles = np.arange(angle_start, angle_end + 1e-6, angle_step)
        readings = []

        for a in angles:
            resp = stage.move_absolute(a)
            time.sleep(0.1)  # allow stage to settle
            v1, _ = rp.acquire(duration_s=measure_duration, samples_per_read=1000)
            readings.append((a, v1))
            print(f"Angle {a:7.3f}° → stage response '{resp}' → V1 = {v1:.6f} V")

        # find minimum
        min_angle, min_v = min(readings, key=lambda x: x[1])
        print("\n→ Minimum CH1 voltage:")
        print(f"   {min_v:.6f} V at angle {min_angle:.3f}°")

    finally:
        stage.close()

if __name__ == "__main__":
    # Adjust these parameters as needed:
    PITAYA_IP    = "10.165.61.248"
    PITAYA_PORT  = 5000
    COM_PORT     = "COM5"

    
    find_min_voltage_angle(
        pitaya_ip=PITAYA_IP,
        pitaya_port=PITAYA_PORT,
        com_port=COM_PORT,
        angle_start=40.0,
        angle_end=50.0,
        angle_step=0.10,
        measure_duration=2.0
    )


## Main code. Data collection with rotational stage

In [1]:
import os
import time
import threading
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import serial
from serial import SerialException
from ipywidgets import (
    Button, HBox, VBox, Layout, Output,
    FloatText, HTML
)
from IPython.display import display

from redpitaya_acquirer import RedPitayaAcquirer

# ── 1) USER PARAMETERS ───────────────────────────────────────────
PITAYA_IP      = '10.165.61.248'
PITAYA_PORT    = 5000

EXCEL_DIR      = r"C:\Users\ruben\OneDrive\Escritorio\Purdue\AONN Research\Experiment data"
FILE_BASE      = "Experiment data"
EXT            = ".xlsx"

# These are now unused except for fixed_offset and backgrounds:
m_var, b_var   = 0.0099, 0.1574
m_out, b_out   = 0.0236, 0.1241

DURATION_S       = 2.0
SAMPLES_PER_READ = 1000
N_MEASUREMENTS   = 60

COM_PORT       = 'COM5'
BAUDRATE       = 9600
INITIAL_ANGLE  = 44.5
STEP_ANGLE     = 0.5
MOVE_DELAY     = 0.1

fixed_offset = 0.0
input_background_voltage = 0.0
output_background_voltage = 0.0

running = False

# Will hold our manual calibrations and voltages:
manual_start_p1 = manual_start_p2 = None
manual_end_p1   = manual_end_p2   = None
v1_start = v2_start = None
v1_end   = v2_end   = None

ALL_COLUMNS = [
    "Voltage 1", "Voltage 2",
    "Power Variable In", "Power Out",
    "Power Fixed", 
    "BG Voltage 1", "BG Voltage 2",
    "Manual P1 Start", "Manual P2 Start",
    "Manual P1 End",   "Manual P2 End"
]

# ── 2) StageController ───────────────────────────────────────────
class StageController:
    def __init__(self, port, baudrate=9600, timeout=1):
        self.ser = serial.Serial(port=port, baudrate=baudrate,
                                 bytesize=serial.EIGHTBITS,
                                 parity=serial.PARITY_NONE,
                                 stopbits=serial.STOPBITS_ONE,
                                 timeout=timeout)
        self.ser.reset_input_buffer()
        self.ser.reset_output_buffer()
        time.sleep(0.1)
        self.pulses_per_deg = 262144/360.0

    def move_relative(self, deg):
        pulses = int(deg*self.pulses_per_deg)
        cmd = f'0mr{pulses:08X}\r'.encode('ascii')
        self.ser.write(cmd)
        return self.ser.readline().decode('ascii').strip()

    def move_absolute(self, deg):
        pulses = int(deg*self.pulses_per_deg)
        cmd = f'0ma{pulses:08X}\r'.encode('ascii')
        self.ser.write(cmd)
        return self.ser.readline().decode('ascii').strip()

    def close(self):
        try: self.ser.close()
        except: pass

# ── 3) Excel setup ───────────────────────────────────────────────
def get_unique_filepath():
    i = 1
    while True:
        name = f"{FILE_BASE}{'' if i==1 else i}{EXT}"
        path = os.path.join(EXCEL_DIR, name)
        if not os.path.exists(path):
            return path
        i += 1

EXCEL_PATH = get_unique_filepath()
if not os.path.exists(EXCEL_PATH):
    pd.DataFrame(columns=ALL_COLUMNS).to_excel(EXCEL_PATH, index=False)
    print(f"Created new data file:\n  {EXCEL_PATH}")

# ── 4) Instruments ────────────────────────────────────────────────
acquirer = RedPitayaAcquirer(ip=PITAYA_IP, port=PITAYA_PORT,
                             decimation=1024, avg_off=True)
try:
    stage = StageController(port=COM_PORT, baudrate=BAUDRATE)
    stage_available = True
except SerialException as e:
    stage_available = False
    stage = None
    print(f"⚠️ Could not open {COM_PORT}: {e}")

file_lock = threading.Lock()
out       = Output()

# ── 5) Manual‐entry UI ─────────────────────────────────────────────
manual_event = threading.Event()
manual_vals  = {}
manual_prompt = HTML("<b>Enter manual P1,P2:</b>")
manual_p1     = FloatText(description="P1:", layout=Layout(width="120px"))
manual_p2     = FloatText(description="P2:", layout=Layout(width="120px"))
manual_enter  = Button(description="Enter", layout=Layout(width="80px"))

def on_manual_enter(_):
    manual_vals['p1'] = manual_p1.value
    manual_vals['p2'] = manual_p2.value
    manual_event.set()
    manual_box.layout.display = 'none'

manual_enter.on_click(on_manual_enter)
manual_box = VBox([manual_prompt, HBox([manual_p1, manual_p2, manual_enter])])
manual_box.layout.display = 'none'

# ── 6) Safe write helper ──────────────────────────────────────────
def safe_write_row(row_dict, overwrite_idx=None):
    while True:
        try:
            with file_lock:
                df = pd.read_excel(EXCEL_PATH)
                # append if no idx, else update only cols in row_dict
                if overwrite_idx is None:
                    row = {c: row_dict.get(c, np.nan) for c in ALL_COLUMNS}
                    df.loc[len(df)] = row
                else:
                    if overwrite_idx >= len(df):
                        for _ in range(len(df), overwrite_idx+1):
                            df.loc[len(df)] = {c: np.nan for c in ALL_COLUMNS}
                    for c,v in row_dict.items():
                        df.at[overwrite_idx, c] = v
                df.to_excel(EXCEL_PATH, index=False)
            return
        except PermissionError:
            with out:
                print("⚠️ Excel busy… retrying in 1 s")
            time.sleep(1)

# ── 7) Single measurement (only raw volts) ───────────────────────
def record_voltages(idx):
    V1, V2 = acquirer.acquire(duration_s=DURATION_S,
                              samples_per_read=SAMPLES_PER_READ)
    safe_write_row({
        "Voltage 1": V1,
        "Voltage 2": V2
    }, overwrite_idx=idx)
    
    with out:
        out.clear_output(wait=True)
        print(f"Row {idx+2}: V1={V1:.4f}, V2={V2:.4f}")
    return V1, V2

# ── 8) N‐loop with manual inputs & final slope fit ───────────────
def measure_n_loop(n, start_btn):
    global running
    global manual_start_p1, manual_start_p2, manual_end_p1, manual_end_p2
    global v1_start, v2_start, v1_end, v2_end

    try:
        # move to initial angle
        if stage_available:
            resp = stage.move_absolute(INITIAL_ANGLE)
            with out:
                out.clear_output(wait=True)
                print(f"Stage → {INITIAL_ANGLE:.2f}° : {resp}")
            time.sleep(MOVE_DELAY)

        # 1) FIRST: record voltages, then manual start
        v1_start, v2_start = record_voltages(0)
        manual_box.layout.display = None
        manual_event.clear()
        manual_event.wait()
        manual_start_p1 = manual_vals['p1']
        manual_start_p2 = manual_vals['p2']
        safe_write_row({
            "Manual P1 Start": manual_start_p1,
            "Manual P2 Start": manual_start_p2
        }, overwrite_idx=0)

        # 2) intermediate measurements
        for i in range(1, n-1):
            if not running: break
            if stage_available:
                stage.move_relative(STEP_ANGLE)
                time.sleep(MOVE_DELAY)
            record_voltages(i)

        # 3) FINAL: move, record, then manual end
        if running:
            if stage_available:
                stage.move_relative(STEP_ANGLE)
                time.sleep(MOVE_DELAY)
            v1_end, v2_end = record_voltages(n-1)
            manual_box.layout.display = None
            manual_event.clear()
            manual_event.wait()
            manual_end_p1 = manual_vals['p1']
            manual_end_p2 = manual_vals['p2']
            safe_write_row({
                "Manual P1 End": manual_end_p1,
                "Manual P2 End": manual_end_p2
            }, overwrite_idx=n-1)

        # 4) compute linear fits & recalculate powers
        slope1 = (manual_end_p1 - manual_start_p1) / (v1_end - v1_start)
        intc1  = manual_start_p1 - slope1 * v1_start
        slope2 = (manual_end_p2 - manual_start_p2) / (v2_end - v2_start)
        intc2  = manual_start_p2 - slope2 * v2_start

        df = pd.read_excel(EXCEL_PATH)
        for i in range(n):
            V1 = df.at[i, "Voltage 1"]
            V2 = df.at[i, "Voltage 2"]
            df.at[i, "Power Variable In"] = slope1 * V1 + intc1
            df.at[i, "Power Out"]         = slope2 * V2 + intc2
        df.to_excel(EXCEL_PATH, index=False)

        with out:
            print("Recalculated power columns with manual fit:")
            print(f" • PD1 slope={slope1:.4f}, intc={intc1:.4f}")
            print(f" • PD2 slope={slope2:.4f}, intc={intc2:.4f}")

    finally:
        if stage_available:
            stage.close()
        running = False
        start_btn.description = "Start"
        with out:
            print("Done; stage closed.")

# ── 9) Button callbacks ──────────────────────────────────────────
def on_fixed_clicked(btn):
    global fixed_offset
    btn.description = "Measuring…"
    V1,_ = acquirer.acquire(duration_s=DURATION_S, samples_per_read=SAMPLES_PER_READ)
    fixed_offset = V1
    safe_write_row({"Voltage Fixed": fixed_offset}, overwrite_idx=0)
    with out:
        out.clear_output(wait=True)
        print(f"Fixed offset in E2: {fixed_offset:.4f}")
    btn.description = "Fixed Laser"

def on_background_clicked(btn):
    btn.description = "Measuring…"
    V1, V2 = acquirer.acquire(duration_s=DURATION_S, samples_per_read=SAMPLES_PER_READ)
    safe_write_row({"BG Voltage 1": V1, "BG Voltage 2": V2}, overwrite_idx=0)
    with out:
        out.clear_output(wait=True)
        print(f"Background F2,G2: V1={V1:.4f}, V2={V2:.4f}")
    btn.description = "Measure Background"

def on_start_clicked(btn):
    global running, stage
    if not running:
        running = True
        btn.description = "Running…"
        if stage_available and not stage.ser.is_open:
            try: stage = StageController(port=COM_PORT, baudrate=BAUDRATE)
            except SerialException as e:
                with out: print(f"⚠️ Cannot reopen {COM_PORT}: {e}")
                running = False; btn.description="Start"; return
        threading.Thread(target=measure_n_loop, args=(N_MEASUREMENTS, btn), daemon=True).start()
    else:
        with out: print("Already running.")

def on_stop_clicked(btn):
    global running
    running = False
    with out:
        out.clear_output(wait=True)
        print("Stop requested.")
    btn.description = "Start"

def on_plot_clicked(btn):
    with out:
        out.clear_output(wait=True)
        df = pd.read_excel(EXCEL_PATH)
        plt.figure()
        plt.plot(df["Power Variable In"], df["Power Out"], 'o-')
        plt.show()
    btn.description = "Plot Variable vs Output"

# ── 10) Build UI ─────────────────────────────────────────────────
fixed_btn      = Button(description="Fixed Laser",        layout=Layout(width="140px"))
background_btn = Button(description="Measure Background", layout=Layout(width="160px"))
start_btn      = Button(description="Start",              layout=Layout(width="100px"))
stop_btn       = Button(description="Stop",               layout=Layout(width="100px"))
plot_btn       = Button(description="Plot Variable vs Output", layout=Layout(width="200px"))

fixed_btn.on_click(on_fixed_clicked)
background_btn.on_click(on_background_clicked)
start_btn.on_click(on_start_clicked)
stop_btn.on_click(on_stop_clicked)
plot_btn.on_click(on_plot_clicked)

display(VBox([
    HBox([fixed_btn, background_btn, start_btn, stop_btn, plot_btn]),
    manual_box,
    out
]))


Created new data file:
  C:\Users\ruben\OneDrive\Escritorio\Purdue\AONN Research\Experiment data\Experiment data27.xlsx
Connected to Red Pitaya at 10.165.61.248:5000


## Code for plotting the graphs of different files

In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

def parse_indices(s: str):
    """
    Parse strings like "1,3,5" or "2-4" into a sorted list of ints.
    """
    parts = re.split(r'\s*,\s*', s.strip())
    nums = set()
    for p in parts:
        if '-' in p:
            a,b = p.split('-',1)
            nums.update(range(int(a), int(b)+1))
        else:
            nums.add(int(p))
    return sorted(nums)

def make_filename(base: str, idx: int, ext: str):
    """
    idx==1 → base + ext
    idx>1  → base + str(idx) + ext
    """
    suffix = '' if idx == 1 else str(idx)
    return f"{base}{suffix}{ext}"

def plot_columns_c_d(excel_dir, file_base, indices, ext='.xlsx'):
    plt.figure()
    for idx in indices:
        fname = make_filename(file_base, idx, ext)
        path = os.path.join(excel_dir, fname)
        if not os.path.exists(path):
            print(f"⚠️  File not found: {path}")
            continue

        df = pd.read_excel(path)
        # column C → index 2, column D → index 3
        x = df.iloc[:, 2]
        y = df.iloc[:, 3]
        plt.plot(x, y, marker='o', label=f"{fname}")

    plt.xlabel("Power before the cell (uW)")
    plt.ylabel("Power after the cell (uW)")
    plt.title("ReLU: self-nonlinearity of channel 1")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # ←── adjust these if needed:
    EXCEL_DIR = r"C:\Users\ruben\OneDrive\Escritorio\Purdue\AONN Research\Experiment data"
    FILE_BASE = "Experiment data"
    EXT       = ".xlsx"

    inp = input("Enter file indices to plot (e.g. 1,3-5): ")
    indices = parse_indices(inp)
    if not indices:
        print("No valid indices provided; exiting.")
    else:
        print(f"Plotting files: {indices}")
        plot_columns_c_d(EXCEL_DIR, FILE_BASE, indices, EXT)
